우선 그 다음으로 넘어가기 전에, pytorch 기본 문법을 잠깐 보고 갈 것이다.

직접 아무것도 안 보고 짤 정도는 아니더라도, 적어도 읽고 얘가 뭔 말을 하는건지는 이해할 수 있어야 하니까, 간단하게 설명을 좀 하고 넘어가겠다.

In [ ]:
import torch

# gpu 를 사용 가능한지, 아닌지는 아래와 같이 확인 가능.
print(torch.cuda.is_available())

# torch 는 모든 데이터를 tensor 로 처리한다.
x = torch.tensor([1.0, 2.0, 3.0])
print(type(x))
print(x)

True
<class 'torch.Tensor'>
tensor([1., 2., 3.])


In [26]:
'''
  cpu 에서 처리한 데이터를 gpu 로 명시적으로 옮겨줘야 하기때문에
  gpu 를 사용중이라면 to("cuda:장비번호") 와 같은 식으로 데이터를 옮겨주어야 한다.
  참고로 같은 장비에 있지 않다면 연산이 불가능하다.
'''

# print(torch.cuda.is_available()) -> false 였다면 아래의 명령어는 동작하지 않는다.
if torch.cuda.is_available():
    x = x.to("cuda")
print(x)

y = torch.tensor([1.0, 2.0, 3.0])

# 아래와 같이 y를 만들면, 초기화할 때 바로 gpu 에 만들 수 있다. 이렇게 만들면 오류가 발생하지 않는다.
# y = torch.tensor([1.0, 2.0, 3.0], device="cuda")

print(x + y)

tensor([0.], device='cuda:0')


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [17]:
# pytorch 는 모델 코드를 작성하면, 계산 그래프를 자동으로 만들고, backward() 를 호출하면
# backpropagation 을 자동으로 계산한다.

# requires_grad = True 라면 Loss에 연관되어 있는 값 들일때 Loss을 backward 하면
# 자동으로 미분 계산에 들어가고,
# False 라면 미분 계산에 들어가지 않는다 (Freeze 된다.)
w = torch.tensor(0.0, requires_grad=True)
b = torch.tensor(0.0, requires_grad=True)

# 이 시점에선 미분값이 존재하지 않는다.
print("w.gradient: ", w.grad, "| b.gradient: ", b.grad)

y = w * 3 + b
L = (y - 5) ** 2

# dy/dw = 3, dy/db = 1
# dL/dy = 2(y-5)
# 알고싶은게 dL/dw 와 dL/db

'''
(편의상 d 를 편미분 기호로 사용 함.)
dL/dw = dL/dy * dy/dw
dL/dy 와 dy/dw 는 이미 알고 있다. 각각 2(y-5) 와 3
즉 dL/dw 는 6(y-5)

마찬가지로
dL/db = dL/dy * dy/db
dy/db = 1 이고 dL/dy 는 2(y-5)

dL/db = 2(y-5)

결과값인 y가 0이 나왔기 때문에 (w=0, b=0 넣고 y = 3w + b 계산하면 y=0 이다.)
L은 25 가 나오고
각 미분값은 w.grad = -30, b.grad = -10 이 나와야 한다.

저 값들을 Learning Rate 와 곱해서 그 값만큼 이동시키면, 조금 더 정답에 가까워 지게 된다.

그래서 Loss 를 최소화 시키는 방향으로 Loss 함수를 미분해서 이동시키면 Loss 가 줄어드는 방향으로 이동하는 것이다.

그렇기 때문에 Loss 가 학습을 결정하는 가장 중요한 요소!
'''

# gradient 를 여기서 계산한다.
L.backward()

print()
# 결과값
print("y:", y.item())
print("L:", L.item())

# 이제 미분값이 존재한다.
print()
print("w.gradient: ", w.grad, "| b.gradient: ", b.grad)


# Learning Rate 를 작은 값을 써야, 천천히 이동하면서 안정적으로 수렴 가능하다.
# LR 이 크면, 오히려 왔다갔다 하면서 발산 해 버린다.
# 해당 예제에서도 lr 을 0.1 을 쓰게 되면, L이 그대로 25 가 나오게 된다. 미분값도 얘네가 미친 영향이 동일해서, 값도 변하지 않는다.
# 0.1 보다 크면 발산한다. (Over Shooting)
# 0.05 보다 크고, 0.1 보다 작으면 감쇠 진동하면서 수렴한다.(lr=0.05 일때 한번에 정답을 맞춘다. L2 가 0이 되는 값.)
'''
| 스텝 | (w, b) | y  | Loss |
| -- | ------ | -- | ---- |
| 0  | (0, 0) | 0  | 25   |
| 1  | (3, 1) | 10 | 25   |
| 2  | (0, 0) | 0  | 25   |
| 3  | (3, 1) | 10 | 25   |
... (이하 중략)
'''
lr = 0.01

# 미분 없이 값만 수정한다는 뜻.
# loss 에서 나온 값을 미분한 값만큼 "빼 주어야 한다."
with torch.no_grad():
    w -= lr * w.grad
    b -= lr * b.grad

# gradient 는 누적되므로, 다시 학습하려면 미분값을 초기화해야 한다
w.grad = None
b.grad = None

# 업데이트 후 forward 계산
y2 = w * 3 + b
L2 = (y2 - 5) ** 2

print("\n업데이트 후 다시 계산한 결과:")
print("updated w:", w.item(), "| updated b:", b.item())
print("y2:", y2.item())
# L2 가 줄었다.
print("L2:", L2.item())

w.gradient:  None | b.gradient:  None

y: 0.0
L: 25.0

w.gradient:  tensor(-30.) | b.gradient:  tensor(-10.)

업데이트 후 다시 계산한 결과:
updated w: 0.29999998211860657 | updated b: 0.09999999403953552
y2: 1.0
L2: 16.0


In [ ]:
# Pytorch 에서 모든 모델은 nn.Module 을 상속 받아서 만든다.
# 이렇게 만들어야 모델이 안에 있는 모든 파라미터들을 자동으로 관리 할 수 있다.
# 그 외에도 아래 있는 모든 기능을 제공한다.
'''
PyTorch에서 nn.Module은 다음 기능을 제공함:
내부에 포함된 모든 nn.Parameter (즉 requires_grad=True인 tensor)를 자동으로 등록
내부에 포함된 모든 서브 모듈(nn.Linear, nn.Conv2d 등)을 자동으로 등록
.parameters(), .named_parameters() 로 파라미터 모으는 기능 제공
.to(device) 를 하면 내부 파라미터 전부를 GPU/CPU로 자동 이동
.eval() / .train() 같은 mode-switch 기능 제공
.state_dict() / .load_state_dict() 로 저장/로드 가능
'''
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        ### Multi Perceptron Layer 에서 입력 차원이 784 차원 / 출력차원이 128 차원 이라는 것이다.
        self.fc = nn.Linear(784, 128)

    ### forward 는 순방향 전파. 그냥 계산하는 과정이다.
    def forward(self, x):
        return self.fc(x)

In [ ]:
# Pytorch 는 Activation 도 nn.module 형태로 제공한다.
'''
nn.ReLU()
nn.Sigmoid()
nn.Tanh()
와 같다.
'''
import torch.nn.functional as F

x = torch.tensor([1.0])
print(x) # 1
x = F.relu(x)
print(x) # 1
x = torch.tensor([-1.0])
x = F.relu(x) # 0
print(x)

tensor([1.])
tensor([1.])
tensor([0.])


In [ ]:
# 기본적으로 많이 사용하는 함수는 nn 내에 정의가 되어 있다.
# 새로운 Loss 의 경우 종종 직접 구현 해야 할 필요가 있다.

# MSE = Mean Square Error
criterion = nn.MSELoss()

# 아래와 같이 쓸 수 있다.
'''
loss = criterion(pred, target)
'''

In [ ]:
# Optimizer는 Gradient 를 보고 weight 를 어떻게 업데이트 할지 정한다.
# Adam 을 많이 사용하나, 다른걸 쓸 때도 있으니 그럴땐 공식 document 를 찾아보면 좋다.
# 보통 논문에는 어떤 Optimizer 를 썼는지, Optimizer 의 Parameter 까지 같이 제공한다.

'''
# 아까는 Leraning Rate 를 직접 곱했는데, 실제로는 직접 곱하지 않는다.
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# gradient 계산
loss.backward()    
# 가중치 업데이트
optimizer.step()   
# gradient 초기화, 위에서 하나짜리 업데이트 할 때는 직접 초기화 했지만, 실제로는 이렇게 초기화 한다.
optimizer.zero_grad()
'''

이 외에 필요한 Dataset 읽기 같은 경우는 필요할 때마다 주석으로 언급하도록 하겠다.